In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 48601
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59437
INFO:root:(('127.0.0.1', 48601)) Conexión aceptada con ('127.0.0.1', 59437)
INFO:root:(('127.0.0.1', 48601)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 48601)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 48601)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 59437)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 59437)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 59437)) Model initialized
INFO:root:(('127.0.0.1', 48601)) Broadcasting Number of Samples...
[('127.0.0.1', 59437), ('127.0.0.1', 59437), ('127.0.0.1', 59437)]
INFO:root:(('127.0.0.1', 48601)) Evaluating...
aun no implementao


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

[('127.0.0.1', 48601), ('127.0.0.1', 48601), ('127.0.0.1', 48601)]
INFO:root:(('127.0.0.1', 59437)) Evaluating...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

INFO:root:(('127.0.0.1', 48601)) Evaluating...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 59437)) Evaluated. Losss: 0.07928825169801712, Metric: 0.953066885471344. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 48601)) Evaluated. Losss: 0.07928825169801712, Metric: 0.9548781514167786. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 59437)) Broadcasting metrics to 1 clients.
INFO:root:(('127.0.0.1', 48601)) Broadcasting metrics to 1 clients.
METRICS RECEIVED ('127.0.0.1:59437', 2, 0.07928825169801712, 0.953066885471344)
INFO:root:(('127.0.0.1', 59437)) Finish!!.
METRICS RECEIVED ('127.0.0.1:48601', 2, 0.07928825169801712, 0.9548781514167786)
INFO:root:(('127.0.0.1', 48601)) Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 48601 y desconectándose de 1 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 48601)
DEBUG:root:Closed connection: ('127.0.0.1', 59437)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 59437 y desconectándose de 0 nodos


In [1]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 58369
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59757
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 33029
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 41593
INFO:root:(('127.0.0.1', 58369)) Conexión aceptada con ('127.0.0.1', 59757)
INFO:root:(('127.0.0.1', 59757)) Conexión aceptada con ('127.0.0.1', 33029)
INFO:root:(('127.0.0.1', 33029)) Conexión aceptada con ('127.0.0.1', 58369)
INFO:root:(('127.0.0.1', 33029)) Conexión aceptada con ('127.0.0.1', 41593)
INFO:root:(('127.0.0.1', 41593)) Conexión aceptada con ('127.0.0.1', 58369)
INFO:root:(('127.0.0.1', 41593)) Conexión aceptada con ('127.0.0.1', 59757)
INFO:root:(('127.0.0.1', 58369)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 58369)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 58369)) Broadcasting model to 3 clients. (size: 548864 bytes)
INFO:root:

GPU available: False, used: False


DEBUG:root:Closed connection: ('127.0.0.1', 59757)
aun no implementao
DEBUG:root:Closed connection: ('127.0.0.1', 59757)


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


DEBUG:root:Closed connection: ('127.0.0.1', 59757)


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 39023)) Evaluated. Losss: 2.301456928253174, Metric: 0.15053437650203705. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 37305)) Evaluated. Losss: 2.301456928253174, Metric: 0.138490691781044. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 39023)) Broadcasting metrics to 2 clients.
INFO:root:(('127.0.0.1', 41229)) Evaluated. Losss: 2.301456928253174, Metric: 0.14942456781864166. (Check tensorboard for more info)
METRICS RECEIVED ('127.0.0.1:39023', 2, 2.301456928253174, 0.15053437650203705)
METRICS RECEIVED ('127.0.0.1:39023', 2, 2.301456928253174, 0.15053437650203705)
INFO:root:(('127.0.0.1', 37305)) Broadcasting metrics to 2 clients.
INFO:root:(('127.0.0.1', 39023)) Finish!!.
METRICS RECEIVED ('127.0.0.1:37305', 2, 2.301456928253174, 0.138490691781044)
METRICS RECEIVED ('127.0.0.1:37305', 2, 2.301456928253174, 0.138490691781044)
INFO:root:(('127.0.0.1', 41229)) Broadcasting metrics to 2 clients.
INFO:root:(('127.0.0.1', 37305)) Finish!

In [2]:
#import threading
for thread in threading.enumerate(): 
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
import sys

def get_thread_position(thread):
    frame = sys._current_frames().get(thread.ident, None)
    if frame:
        return frame.f_code.co_filename, frame.f_code.co_name, frame.f_code.co_firstlineno

get_thread_position(threading.enumerate()[-1])

('/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py',
 '__on_round_finished',
 260)

In [1]:
from p2pfl.const import HEARTBEAT_FREC, SOCKET_TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
from test.fixtures import two_nodes, four_nodes


import threading
for thread in threading.enumerate(): 
    print(thread.name)
    
n1 = Node(MLP(),MnistFederatedDM())
n2 = Node(MLP(),MnistFederatedDM())
print("\n")
def test_bad_binary_model():

    n1.start()
    n2.start()

    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) 

    # Start Learning
    n1.set_start_learning(rounds=2,epochs=0)
 
    # Adding noise to the buffer
    for _ in range(200):self.neightboors
        n1.neightboors[0].param_bufffer += "noise".encode("utf-8")
    
    while not n1.round is None and not n2.round is None:
        time.sleep(0.1)
        
    n1.stop()
    n2.stop()
    
test_bad_binary_model()

time.sleep(10)

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n")



ModuleNotFoundError: No module named 'p2pfl.const'

In [4]:

for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
print(n1.round)

None


In [4]:
from p2pfl.settings import Settings

Settings.AGREGATION_TIEMOUT = 1

In [5]:
"0.1545318067073822".isdecimal()

False

In [4]:
message = ['METRICS', '0', '2.301456928253174', '0.13700607419013977']
print(int(message[1]), float(message[2]), float(message[3]))

0 2.301456928253174 0.13700607419013977


In [11]:
import random
import math
a = [1,2,3,4,5,6,7,8,9,10]

random.choices(a, k=3)

[5, 8, 2]

In [13]:
[random.randint(0,1000),math.floor(random.randint(0,1000)/2),math.floor(random.randint(0,1000)/4)]

[300, 298, 15]

In [15]:
"".join(("1","2"))

'12'

In [20]:
def build_vote_train_set_msg(candidates, weights):
    aux = ""
    for i in range(len(candidates)):
        aux = aux + " " + " ".join(candidates[i]) + " " + str(weights[i])
    return ("CommunicationProtocol.VOTE_TRAIN_SET" + aux + "\n").encode("utf-8")

candidates = [("host","port"),("host","port"),("host","port")]
weights = [random.randint(0,1000),math.floor(random.randint(0,1000)/2),math.floor(random.randint(0,1000)/4)]

build_vote_train_set_msg(candidates,weights)

b'CommunicationProtocol.VOTE_TRAIN_SET host port 443 host port 194 host port 245\n'

In [2]:
[str(x) for x in ("1",2)]

['1', '2']

In [1]:
import random
random.choices([], k=3)

IndexError: list index out of range